In [18]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


from pyspark.sql import SparkSession
from pyspark.sql.functions import asc, lit

from pyspark.sql.functions import col, \
    from_unixtime, to_date, date_add, when, monotonically_increasing_id, window, mean, count
from pyspark.sql.types import TimestampType


In [19]:
# Create a Spark session
spark = SparkSession.builder \
    .appName("Mean Computer Generator") \
    .getOrCreate()

In [20]:
spark

In [21]:
FLIGHT_ICAO = 'a7124d'

In [22]:
flights_file_path = os.path.join('dataset', 'raw', FLIGHT_ICAO)

# flights_file_path = os.path.join('dataset', 'raw', FLIGHT_ICAO , f'{FLIGHT_ICAO}_0.parquet')

read_flights = spark.read.parquet(flights_file_path, header=True)

In [23]:
def get_values_from_selected_columns(flight_data):

    #time_diff|      time|icao24|              lat|               lon|          velocity|           heading|          vertrate|callsign|onground|alert|  spi|squawk|      baroaltitude|       geoaltitude|   lastposupdate|     lastcontact|flight_number
    selected_cols = flight_data.select('time', 'icao24', 'lat', 'lon', 'velocity', 'heading', 'vertrate',  \
                                       'baroaltitude', 'geoaltitude', 'lastposupdate', 'lastcontact' , \
                                        'velocity', 'time_diff', 'dropout_length') \
                        .orderBy(asc('time'))
    
    selected_cols = selected_cols.withColumn("datetime", from_unixtime("time"))

    return selected_cols

In [24]:
flight_df = get_values_from_selected_columns(read_flights)

In [25]:
flight_df.show(3)

+----------+------+------------------+------------------+------------------+------------------+------------------+------------+------------------+----------------+----------------+------------------+---------+------------------+-------------------+
|      time|icao24|               lat|               lon|          velocity|           heading|          vertrate|baroaltitude|       geoaltitude|   lastposupdate|     lastcontact|          velocity|time_diff|    dropout_length|           datetime|
+----------+------+------------------+------------------+------------------+------------------+------------------+------------+------------------+----------------+----------------+------------------+---------+------------------+-------------------+
|1645384743|a7124d|30.662422826734637|  -97.784912109375|              null|              null|              null|     1127.76|              null|1.645384742385E9|1.645384742385E9|              null|     null|              null|2022-02-20 13:19:03|
|164

In [26]:
flight_df.count()

3100

#### Preprocessing

##### 1. Duplicates

In [27]:
updated_flight_df = flight_df.groupBy(flight_df.columns)\
    .agg(count('*').alias("count"))\
        .filter(col("count") > 1)

updated_flight_df.show()

+----+------+---+---+--------+-------+--------+------------+-----------+-------------+-----------+--------+---------+--------------+--------+-----+
|time|icao24|lat|lon|velocity|heading|vertrate|baroaltitude|geoaltitude|lastposupdate|lastcontact|velocity|time_diff|dropout_length|datetime|count|
+----+------+---+---+--------+-------+--------+------------+-----------+-------------+-----------+--------+---------+--------------+--------+-----+
+----+------+---+---+--------+-------+--------+------------+-----------+-------------+-----------+--------+---------+--------------+--------+-----+



In [28]:
updated_flight_df = flight_df.dropDuplicates()

In [29]:
updated_flight_df.show()

+----------+------+------------------+------------------+------------------+------------------+-------------------+------------------+-----------+----------------+----------------+------------------+---------+-------------------+-------------------+
|      time|icao24|               lat|               lon|          velocity|           heading|           vertrate|      baroaltitude|geoaltitude|   lastposupdate|     lastcontact|          velocity|time_diff|     dropout_length|           datetime|
+----------+------+------------------+------------------+------------------+------------------+-------------------+------------------+-----------+----------------+----------------+------------------+---------+-------------------+-------------------+
|1645384860|a7124d|30.675933837890625| -97.8423533720129| 53.47248830179257| 281.0937230115578|            0.32512|           1318.26|     1333.5|1.645384859624E9|1.645384859624E9| 53.47248830179257|      1.0| 0.6549999713897705|2022-02-20 13:21:00|


##### 2. Drop NAs

In [30]:
updated_flight_df = updated_flight_df.na.drop().show()

+----------+------+------------------+------------------+------------------+------------------+-------------------+------------------+-----------+----------------+----------------+------------------+---------+-------------------+-------------------+
|      time|icao24|               lat|               lon|          velocity|           heading|           vertrate|      baroaltitude|geoaltitude|   lastposupdate|     lastcontact|          velocity|time_diff|     dropout_length|           datetime|
+----------+------+------------------+------------------+------------------+------------------+-------------------+------------------+-----------+----------------+----------------+------------------+---------+-------------------+-------------------+
|1645384860|a7124d|30.675933837890625| -97.8423533720129| 53.47248830179257| 281.0937230115578|            0.32512|           1318.26|     1333.5|1.645384859624E9|1.645384859624E9| 53.47248830179257|      1.0| 0.6549999713897705|2022-02-20 13:21:00|


#### Calculating Mean

In [31]:
# Convert the timestamp column to a TimestampType
mean_calculated_df = flight_df.withColumn("datetime", flight_df["datetime"].cast(TimestampType()))

window_size = "2 minutes"

mean_calculated_df = mean_calculated_df.withColumn("window", window("datetime", window_size)).select('icao24', 'window', 'geoaltitude', 'velocity')

mean_calculated_df.show(5, truncate=False)

+------+------------------------------------------+------------------+------------------+
|icao24|window                                    |geoaltitude       |velocity          |
+------+------------------------------------------+------------------+------------------+
|a7124d|{2022-02-20 13:18:00, 2022-02-20 13:20:00}|null              |null              |
|a7124d|{2022-02-20 13:18:00, 2022-02-20 13:20:00}|1165.8600000000001|41.058948447279064|
|a7124d|{2022-02-20 13:18:00, 2022-02-20 13:20:00}|1165.8600000000001|41.197297370691295|
|a7124d|{2022-02-20 13:18:00, 2022-02-20 13:20:00}|1165.8600000000001|41.197297370691295|
|a7124d|{2022-02-20 13:18:00, 2022-02-20 13:20:00}|1173.48           |42.137328924285455|
+------+------------------------------------------+------------------+------------------+
only showing top 5 rows



In [32]:
agg_df = mean_calculated_df.groupBy("window")\
        .agg(mean(flight_df['geoaltitude']).alias("mean_geoaltitude"),
             mean(flight_df['velocity']).alias("mean_velocity")).orderBy(asc('window'))


agg_df.show(truncate=False)

+------------------------------------------+------------------+------------------+
|window                                    |mean_geoaltitude  |mean_velocity     |
+------------------------------------------+------------------+------------------+
|{2022-02-20 13:18:00, 2022-02-20 13:20:00}|1214.1653571428567|47.671538433684525|
|{2022-02-20 13:20:00, 2022-02-20 13:22:00}|1320.863499999999 |53.258583368461274|
|{2022-02-20 13:22:00, 2022-02-20 13:24:00}|1361.1860000000008|57.95205760405555 |
|{2022-02-20 13:24:00, 2022-02-20 13:26:00}|1368.425          |60.52901298236391 |
|{2022-02-20 13:26:00, 2022-02-20 13:28:00}|1354.454999999999 |60.88966417262641 |
|{2022-02-20 13:28:00, 2022-02-20 13:30:00}|1356.0424999999996|60.15135162559008 |
|{2022-02-20 13:30:00, 2022-02-20 13:32:00}|1335.4684999999993|60.16591271413828 |
|{2022-02-20 13:32:00, 2022-02-20 13:34:00}|1341.5644999999977|60.12526130245427 |
|{2022-02-20 13:34:00, 2022-02-20 13:36:00}|1341.1199999999978|60.139361733997255|
|{20

In [33]:
agg_df.count()

29

In [34]:
agg_df.toPandas().to_csv(os.path.join('dataset', 'processed', f'{FLIGHT_ICAO}_all_trips_m2.csv'))